# A* Algorithm

## Preamble (Vishal Thomas 124108347)

**A\*** is a heuristic search algorithm for increasing the efficiency of Dijkstra’s algorithm for pathfinding and graph traversal. Here heuristic information means to find a suitable or an intuitive function that helps us to find the shortest path between 2 nodes in a graph, efficiently. It is used in gaming applications especially to design the movements of agents when they are aiming to reach a goal. It is used in network connection systems for sending packets between routers, cost effectively. It also helps robots in navigation$^{[1]}$. It uses the heuristic cost of greedy best first search algorithm and exact cost of Dijkstra's algorithm to find the shortest path. It calculates the heuristic distance from node **n** to the end node and exact cost from start node to **n**.
<p><center>$Distance_{new}(start,v) = Heuristic(v,end) + Distance_{old}(start,v)$</center></p>

$Distance_{new}(start,v)$ is the new updated distance between start node and node **v**, $Distance_{old}(start,v)$ is the exact cost of reaching node **v** from start node and $Heuristic(v,end)$ is the heuristic cost of reaching to end node from **v**. After updating the distance in the graph, we will calculate the shortest path.

<h4>Navigation through Maps</h4>
<p>For finding optimal path between 2 cities or places, we can use A* search algorithm to calculate the shortest path between them. Here the heuristic function could be the air distance(euclidean distance) or great-circle distance between 2 cities.$^{[2]}$</p>

<h4>Video Games</h4>
Pathfinding is a crucial part in Video Games. A* is one of the widely used algorithms in games especially when it involves path-finding problems such as moving characters from one grid point to another grid point. Due to the exponential growth in complexity of the games, researchers focused on improving the A* algorithm to meet the demands of modern games. Many of the optimizations include improving heuristic functions, search space(maps), data structures and reducing space complexity.$^{[3]}$

![title](fig1.png)
<p>Fig 1: Screenshot from civilization V game, where it uses hexagonal tiles as search space$^{[3]}$</p>

## Short history (Daniel Sedlov 123120712)

<p>The A* Algorithm was invented by Bertram Raphael (Goldstein, A., 1991, July 25). It was Invented in the early 1970s as part of development of “Shakey the Robot”, a rather uninspired name for an early robot used for navigating rooms autonomously, funded by DARPA (DARPA). <br>
Much like the Boston Dynamics robots making the news today, this robot was supposed to be science fiction turned into reality. While primitive, it navigated essentially the same as modern robots and self-driving cars do today. It had a destination in mind it was trying to reach, and a collection of sensors informing it of occupied spaces, i.e. obstacles. This set the stage for A*.</p>

Shakey the Robot found its path by pointing and heading into the direction of the end point where it wanted to go, and finding the most optimal route around obstacles it encountered by looking left and right around the obstacles to get a sense for its surroundings. [3]
This is in essence how the A* algorithm works today, by imitating the movements and actions of Shakey the robot looking to its destination, and shaking around once it encounters an obstacle. 

## Core

### Implementation (Daniel Sedlov 123120712)

There are many ways to search for a path, each having their own benefits, however the strength of the A* algorithm comes from the guarantee that it finds the shortest path on a grid more efficiently than other algorithms, given a known start and end point. At a high level, it achieves this by first assuming there are no obstacles, and simply moving in that direction. 
Once it inevitably encounters an obstacle, it searches around it to find the shortest path. Crucially, it is not greedy. This means that when it finds a short path around the obstacle, it doesn’t lock in that path, so if later it finds too many other detours, it is willing to go back and take a path that is longer in the short term, to find a path shorter overall.

To explore this in more detail, we will set up our environment in a grid.

We assume that all squares in this grid can be in one of 3 states: open, closed, or not considered.
Open means that we have not yet explored that cell but it in the queue of options. 
Closed means we have explored it, and do not need to check it anymore.
Not considered simply means neither open nor closed, we have not explored it yet, and we’re not really thinking about it yet either.

For any of the algorithm to make sense, we have to prioritise the order in which we explore the open cells. This means that unlike a simple breadth search, we need a way to say “*this cell to the right has more promise than the cell to my left*”.
Importantly, we also need to make sure that we have a mechanism to return to earlier points in the path, unlike a greedy pathfinding solution. [5]

To break this down a bit more and in plain language, lets assign each cell to have 3 values:

g: The distance to the goal as the crow flies
h: The distance from the start as the crow flies
cost: the sum of the 2

The basic loop is as follows:

Step 1
If end is not found, explore the next cell by finding the surrounding cells that are not closed, calculate the g, h, and cost values, set them to open if they are not already (at the beginning, the next cell is just the starting cell)

Step 2
Order the open cells by cost

Step 3
Explore all open cells in order of cost
If there are no ties, go to Step 1

Step 4
If there is a tie, explore the cell with a lower h
If there is still a tie, then select one at random
Go to Step 1


## Demo Daniel Sedlov (123120712)

To illustrate the process, I am taking inspiration from [Reb Block Games' example](https://www.redblobgames.com/pathfinding/a-star/implementation.html)

#### Imports

In [1]:
from __future__ import annotations
from typing import Protocol, Iterator, Tuple, TypeVar, Optional
import heapq

#### Classes

First up is the square grid class which simply creates a grid of a certain width and height and structures it like a network.
This allows us to code everything as it if was a network, but displaying it like it was a grid. This makes the code very adapable to any other more realistic network type situations.

Note this is implementation assumes that each cell has only 4 neighbours. it is possible to increase that amount to 8 for example. However that is up to the implementation of the search

In [2]:
class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: list[GridLocation] = []
        self.weights: dict[GridLocation, float] = {} #basically this is cost. this is how we prioritise who is first
    
    #validate coordinates are in bounds
    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    #validate that coords are in a free location
    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls
    
    #get neighbours while filtering out anything that is out of bounds or a wall
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        return self.weights.get(to_node, 1) 


This is a well-known data structure in the world of computer science. It very basically just stores elements in a heap, and pops them out in order of a certain priority value (in our case, the cost)

In [3]:
class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, T]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> T:
        return heapq.heappop(self.elements)[1]   

#### Functions



In [4]:
#Converts count into coordinate values
def from_id_width(id, width):
    return (id % width, id // width)

#defines the icons that are used in a drawing
def draw_tile(graph, id, style):
    r = " . "
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " v "
        if y2 == y1 - 1: r = " ^ "
    if 'path' in style and id in style['path']:   r = " @ "
    if 'start' in style and id == style['start']: r = " A "
    if 'goal' in style and id == style['goal']:   r = " Z "
    if id in graph.walls: r = "###"
    return r

#Draws the grid
def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("~~~" * graph.width)    

#finds the distance
def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

#runs the A* pathfinding
def a_star_search(graph, start: Location, goal: Location, steps = -1):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[Location, Optional[Location]] = {}
    cost_so_far: dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    total_cost: dict[Location, float] = {}
    step = steps + 1
    
    while not frontier.empty():
        current: Location = frontier.get()
        if(steps > 0):
            step -= 1
            if(step <= 0):
                break
        
        if current == goal:
            break
            
        #calculate new open cells and prioritises based on cost
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                total_cost[next] = priority
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, total_cost

#Gets path to draw once path is found
def reconstruct_path(came_from: dict[Location, Location],
                     start: Location, goal: Location) -> list[Location]:

    current: Location = goal
    path: list[Location] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse() 
    return path

#### Setup

Sets up enviroment to pathfind in and inital conditions

A = Start
Z = End

In [5]:
DIAGRAM_WALLS = [from_id_width(id, width=30) for id in [21,22,51,52,81,82,93,94,111,112,123,124,133,134,141,142,153,154,163,164,171,172,173,174,175,183,184,193,194,201,202,203,204,205,213,214,223,224,243,244,253,254,273,274,283,284,303,304,313,314,333,334,343,344,373,374,403,404,433,434]]
diagram = SquareGrid(30, 15)
diagram.walls = DIAGRAM_WALLS

start = (1, 8)
goal = (24, 3)

draw_grid(diagram, start=start, goal=goal)

__________________________________________________________________________________________
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  Z  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  A  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

Now we can finally watch how the code finds the best path. It will show all the costs along the way, and display its most optiomal path at the end

In [6]:

for i in range(1,280,10):
    came_from, total_cost = a_star_search(diagram, start, goal, steps = i)
    print()
    draw_grid(diagram, number=total_cost, start=start, goal=goal)
came_from, cost_so_far = a_star_search(diagram, start, goal)
print()
draw_grid(diagram, path=reconstruct_path(came_from, start=start, goal=goal))


__________________________________________________________________________________________
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  Z  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ###### .  .  .  .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  .  . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 .  28 . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 30 A  28###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  30 . ###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

 34 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32###### .  .  .  .  .  .  . 
 32 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30###### .  .  .  .  .  .  . 
 30 28 28###### 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30###### .  Z  .  .  .  .  . 
 30 28 28###### 32 32 32 32 32 32 32 32###### 32 32 32 32 32 32###### .  .  .  .  .  .  . 
 30 28 28###### 34 34 34 .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 30 28 28###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  . ############### .  .  .  . 
 30 28 28###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 30 A  28###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 32 30 30###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 34 32 32###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  34 34###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 

 30 A  28###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 32 30 30###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 34 32 32###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 36 34 34###### .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  36 36 .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

__________________________________________________________________________________________
 36 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34###### .  .  .  .  .  .  . 
 34 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32###### .  .  .  .  .  .  .

 30 28 28###### 36 36 36 36 36 36 36 36###### 36 36 36 36 36 36############### .  .  .  . 
 30 28 28###### 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 30 A  28###### 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 32 30 30###### 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 34 32 32###### 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 36 34 34###### 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 38 36 36 36 36 36 36 36 36 36 36 36 36###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  38 38 38 38 38 38 38 38 38 38 38 . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  . ###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

_________________________________________________________________________________________

 42 40 40 40 40 40 40 40 40 40 40 40 40###### .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

__________________________________________________________________________________________
 36 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34###### .  .  .  .  .  .  . 
 34 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32###### .  .  .  .  .  .  . 
 32 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30###### .  .  .  .  .  .  . 
 30 28 28###### 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30###### .  Z  .  .  .  .  . 
 30 28 28###### 32 32 32 32 32 32 32 32###### 32 32 32 32 32 32###### .  .  .  .  .  .  . 
 30 28 28###### 34 34 34 34 34 34 34 34###### 34 34 34 34 34 34############### .  .  .  . 
 30 28 28###### 36 36 36 36 36 36 36 36###### 36 36 36 36 36 36############### .  .  .  . 
 30 28 28###### 36 36 36 36 36 36 36 36###### 38 38 38 38 38 38 38 38 38 38 40 42 .  .  .

<h2>Demo Vishal Tomas (124108347)</h2>

<p>Here are few examples for demonstrating A* algorithm using <b>networkx</b> and <b>ipywidgets</b> library.</p>
<h4>1. Finding shortest path in the graphs</h4>
<p>Here we can see the comparison between the Dijkstras algorithm and A* search algorithm. It also shows why we need to devise a proper heuristic function for A*, or else Dijkstras algorithm is more suitable for calculating shortest path. The below demonstration is interactive where you can select the number of nodes in the graph using a slider. You can also choose the start node and end node to calculate the shortest path.</p>
<p>The graph used here is watts_strogatz_graph (a random graph generator) with nearest neighbours($k$) as $8$ and probability($p$) of the connectivity between nodes is $0.2$. </p>
<p>Assuming weight of all edges is $1$</p>
<p> Heuristic function used here is L1 norm between current node and end node: $L1_{norm} = ||current\_node - end\_node||$</p>

In [1]:
# Import necessary libraries
import networkx as nx
import time
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

seed = 6020

In [4]:
canvas = []
# Interactive widgets to get value of nodes, start and end node
nodes_slider = widgets.IntSlider(description="nodes: ",max=5000, min=500, step=200, value=1000)
start_node = widgets.IntText(description="Start node: ", value=0)
end_node = widgets.IntText(description="End node: ", value=0)
canvas+=[nodes_slider, start_node, end_node]

output = widgets.Output()

def slider_update(change):
    
    nodes=nodes_slider.get_interact_value()
    start=start_node.get_interact_value() 
    end=end_node.get_interact_value()
    with output:
        # Clear the output window
        output.clear_output(wait=True)
        # Create the random graph with k and p, weights are 1, if its not provided explicitly
        G = nx.watts_strogatz_graph(nodes,8,0.2,seed)
        # Record starting time of process
        start_time = time.process_time_ns()
        # Find Dijkstra's Path
        dij_path = nx.dijkstra_path(G,start,end)
        # Record time when process ends
        end_time = time.process_time_ns()
        print("Time taken to compute Dijkstra's path: ",end_time-start_time)
        # Create graph with paths computed in Dijkstra's algorithm
        dij_graph = nx.Graph([(dij_path[i],dij_path[i+1]) for i in range(len(dij_path)-1)])
        plt.figure(figsize=(6, 4))
        nx.draw(
            dij_graph,
            with_labels=True,
            node_color="lightgreen",
            edge_color="blue",
            node_size=600,
            font_size=10,
        )
        plt.show()
        start_time = time.process_time_ns()
        # Create graph with paths computed in Dijkstra's algorithm
        astar_path = nx.astar_path(G,start,end,lambda a,b: abs(b-a))
        end_time = time.process_time_ns()
        print("Time taken to compute Astar path: ", end_time-start_time)
        # Create graph with paths computed in A* search algorithm
        astar_graph = nx.Graph([(astar_path[i],astar_path[i+1]) for i in range(len(astar_path)-1)])
        plt.figure(figsize=(6, 4))
        nx.draw(
            astar_graph,
            with_labels=True,
            node_color="lightgreen",
            edge_color="blue",
            node_size=600,
            font_size=10,
        )
        plt.show()

nodes_slider.observe(slider_update, names="value")
start_node.observe(slider_update, names="value")
end_node.observe(slider_update, names="value")

display(widgets.VBox(canvas), output)

Output()

<h4>Observation</h4>
<p>As you can see the computation time differs for both algorithm when number of nodes is $2100$, start node is $0$ and end node is $100$. For Dijkstras it took $15625000\space ns$ (nano seconds), and for A* it took $0\space ns$. But the caveat is A* doesn't give you the shortest path whereas Dijkstras gives you the optimal path. Therefore we can conclude that if we don't get the heuristics correct, A* will not work correctly as we imagined.</p>

<h4>2. A* algorithm used in grids</h4>
<p>We will create a grid of size $n\times n$, where you can choose the grid size $n$, start node and end node to calculate shortest path. You can also add blocks in the grid. After placing blocks (if any), start and end nodes, just click <b>calculate</b> button to find the shortest path using A*. After the calculation, you can see green-colored cells in the grid which denotes the shortest path between start and end node</p>

In [5]:
# Initializing
canvas = []
grid = None
grid_size = None
grid_graph = None
cells = None
block = None
start = None
end = None
check_types = {}

# Get grid size
inp_grid_size = widgets.IntText(description='size', value=5)
output = widgets.Output()

canvas+=[inp_grid_size]
# Define heuristics function for a-star search algorithm
def heuristics(a,b):
    x0,y0 = int(a[0]), int(a[1])
    x1,y1 = int(b[0]), int(b[1])
    return ((x1-x0)**2 + (y1-y0)**2)**0.5

def toggler(change):
    owner_name = change.get("owner",{"description": ""}).description
    for key in check_types.keys():
        if key != owner_name:
            check_types[key]["obj"].value = False

def update_grid(widget):
    global start_node, end_node
    if (widget.description == "Calculate"):
        start_time = time.process_time_ns()
        astar_paths = nx.astar_path(grid_graph, start_node, end_node, heuristics)
        end_time = time.process_time_ns()
        with output:
            print("Time to calculate the path: ", end_time - start_time)
        # Highlight paths of astar
        for node in astar_paths[1:-1]:
            i,j = node[0], node[1]
            cells[i].children[j].style.button_color="green"
             
    else:
        # for cells
        x,y = [int(i) for i in widget.description.split(",")]
        cell_type = [val for val in check_types.values() if val["obj"].value]
        if cell_type:
            cell = cell_type[0]
            if cell["ops"] == "remove_cell":
                if cell["color"] == widget.style.button_color:
                    edges = [((x-i,y-j),(x,y)) for (i,j) in [(0,1),(1,0), (0,-1), (-1,0), (1,1),(1,-1), (-1,-1), (-1,1)] if (0<=(x-i)<grid_size) and (0<=(y-j)<grid_size)]
                    grid_graph.add_edges_from(edges)
                    widget.style.button_color = "lightgrey"
                    return
                grid_graph.remove_node((x,y))
            elif cell["ops"] == "set_start_node":
                start_node = (x,y)
            elif cell["ops"] == "set_end_node":
                end_node = (x,y)
            widget.style.button_color = cell["color"]
            return
        widget.style.button_color = "lightgrey"


def reset(_):
    create_grid()

def create_grid(change=None):
    global grid_graph, cells, grid_size
    grid_size = inp_grid_size.get_interact_value()
    # Creating graph with edges that is similar to functions used in a grid to move from one cell to another
    grid_graph = nx.grid_graph((grid_size,grid_size))
    for node in grid_graph.nodes():
        x,y = node
        # Adding diagonal connections
        edges = [((x-i,y-j),(x,y)) for (i,j) in [(1,1),(1,-1), (-1,-1), (-1,1)] if (0<=(x-i)<grid_size) and (0<=(y-j)<grid_size)]
        grid_graph.add_edges_from(edges)
    with output:
        output.clear_output(wait=True)
        plot_canvas = []
        cells = []
        for i in range(grid_size):
            row = []
            for j in range(grid_size):
                # Creating button as a cell
                cell = widgets.Button(
                    description=f"{i},{j}",  
                    layout=widgets.Layout(width='20px', height='20px'),
                    style=widgets.ButtonStyle(font_size="10px", button_color="lightgrey")
                    ) 
                cell.on_click(update_grid)
                row.append(cell)
            cells.append(widgets.HBox(row))
        plot_canvas.append(widgets.VBox(cells))
        
        # Blocks
        block = widgets.Checkbox(description="Add Block")
        block.observe(toggler)
        check_types["Add Block"] = {"obj": block, "color": "black", "ops": "remove_cell"}
        # start node
        start_node_inp = widgets.Checkbox(description="Start Node")
        start_node_inp.observe(toggler)
        check_types["Start Node"] = {"obj": start_node_inp, "color": "blue", "ops": "set_start_node"}
        # end node
        end_node_inp = widgets.Checkbox(description="End Node")
        end_node_inp.observe(toggler)
        check_types["End Node"] = {"obj": end_node_inp, "color": "red", "ops": "set_end_node"}
        # Calculate distance
        calc_btn = widgets.Button(description="Calculate")
        calc_btn.on_click(update_grid)
        # Reset entire grid
        reset_btn = widgets.Button(description="Reset")
        reset_btn.on_click(reset)
        # Add it to canvas plot
        plot_canvas .append(widgets.VBox([block, start_node_inp, end_node_inp, calc_btn,reset_btn]))
        display(widgets.HBox(plot_canvas))          

inp_grid_size.observe(create_grid, names="value")
display(widgets.HBox(canvas), output)
create_grid()

Output()

<h4>Observation</h4>
<p>For below state of the grid:</p>
<ol>
    <li>Grid size: $5 \times 5$</li>
    <li>Blocks are placed at: $(2,1), (2,2), (1,2)$</li>
    <li>Start Node: $(0,0)$</li>
    <li>End Node: $(3,3)$</li>
</ol>
<p>The reason for selecting $(1,1)$ as the first cell in the shortest path is because the value of heuristic function evaluated at $(1,1)$ is lesser compared to other cells $(1,0), (0,1)$. </p>
<p>Since $Heuristics((x_0, y_0), (x_1, y_1)) = \sqrt{(x_0 - x_1)^2+(y_0 - y_1)^2}$</p>
<center>$Heuristics((1,1),(3,3)) = 2.83$</center>
<center>$Heuristics((1,0),(3,3)) = 3.61$</center>
<center>$Heuristics((0,1),(3,3)) = 3.61$</center>


<h3>Other Search Algorithms (Vishal Thomas 1234108347)</h4> 

<h5>Dijkstra's Algorithm$^{[4]}$</h5>
<p>An algorithm to find the shortest path in a graph with positive edge weights. It maintains a set of visited and unvisited vertices. It moves to next unvisited node from a node and update the distance if its lower than the preivous. This process continue until the destination vertex is reached.</p>

<h5>Greedy Best First Search$^{[5]}$</h5>
<p>A pathfinding algorithm that works by utilising heuristic function to determine cost effective paths. It identifies the shortest path by evaluating the heuristic function from the current node to the end node. This process is continued until destination node is reached.</p>

<h5>Bellman-Ford Algorithm$^{[6]}$</h5>
<p>It is a shortest path algorithm which works well in negative weighted edges. It updates the shortest distance of node, if a shorter path is found through another node.</p>

<h5>Floyd-Warshall Algorithm$^{[7]}$</h5>
<p>It's the shortest path finding algorithm for all pairs of vertices. For reducing time complexity, it uses Dynamic Programming approach to find the shortest path between vertices, if possible. It doesn’t work for negative cycles, where your sum of edges in a cycle is less than zero.
</p>
<h3>Comparisons (Vishal Thomas 1234108347)</h3> 
<p>For understanding how A* algorithm differs from above mentioned path finding algorithms, we are using certain features to differ them, such as:</p>
<p>
    <li><b>Optimization</b>: Discuss on what area algorithm is optimized for.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: Finding shortest path between single source node and all other nodes based on heuristic function that depends on current node and end node.</li>
        <li><b>Dijkstra's Algorithm</b>: Finding shortest path between single source node and all other nodes with positive weighted edges.</li>
        <li><b>Bellman-Ford Algorithm</b>: Finding shortest path between single source node and all other nodes including negative weighted edges.</li>
        <li><b>Floyd-Warshall Algorithm</b>: Finding shortest path between all pairs of nodes with only positive weighted edges.</li>
        <li><b>A* Algorithm</b>: Finding shortest path between start and end nodes by reducing search space</li>
        </ol>
    </li>
    <li><b>Relaxation</b>: Discuss on how weights are updated in each iteration.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It finds the node having the smallest distance in a greedy approach and updates the distance of its neighbours.</li>
        <li><b>Dijkstra's Algorithm</b>: It finds the node having the smallest distance based on the actual cost and updates the distance of its neighbours.</li>
        <li><b>Bellman-Ford Algorithm, Floyd-Warshall Algorithm</b>: It considers all possible paths to nodes and updates the distance of each node.</li>
        <li><b>A* Algorithm</b>: It chooses the node having minimal sum of distance from start node and heuristic distance from end node. Weights are updated after adding heuristics.</li>
        </ol>
    </li>
    <li><b>Time Complexity</b>: Discuss about the time taken by algorithm to complete. For below, $E$ stands for edges in the graph and $V$ stands for vertices in the graph.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It depends on heuristic function.</li>
        <li><b>Dijkstra's Algorithm</b>: Average is <b>$O(E \times log(V))$</b>, worst is <b>$O(V^2)$</b>. </li>
        <li><b>Bellman-Ford Algorithm</b>: It has a time complexity of <b>$O(VE)$</b>.</li>
        <li><b>Floyd-Warshall Algorithm</b>: It has time complexity of $O(V^3)$.</li>
        <li><b>A* Algorithm</b>: Time complexity depends on heuristic function.</li>
        </ol>
    </li>
    <li><b>Search Techniques</b>: Discuss about the search techniques such as the parameters involved in updating weight functions.$^{[4]}$
        <ol>
        <li><b>Greedy Best First Search</b>: It uses heuristic function to reach to the end node.</li>
        <li><b>Dijkstra's Algorithm, Bellman-Ford Algorithm, Floyd-Warshall Algorithm</b>: It uses the weights initialized in the graph for calculating the shortest path.</li>
            <li><b>A* Algorithm</b>: It uses weights initialised and heuristic function to guide the search to end node</li>
        </ol>
    </li>
</p>

## Variants (Vishal Thomas 124108347)
<h4>Heirarchical Pathfinding A* (HPA*) .$^{[3]}$</h4>
<p>Using hierarchical A*, we can reduce the search space hierarchically. For example, if we are going to Blackrock, Dublin from Kinsale, Cork (as shown in Fig 2.), by using A* implementation we will find all the cities in each county as nodes and find its heuristic function value to compute the shortest path between these two. If we use hierarchical A* algorithm, we can reduce the size of search space by considering its counties as its nodes, not the cities. First travel to the neighbourhood of the starting node i.e. boundary of Cork County. Then find the neighbourhood of the end node which is Dublin. After reaching the neighbourhood, travel to the end node i.e. Blackrock. As you can see we have eliminated the cities(nodes) that are not part of Cork and Dublin, and thus reduced search space dramatically resulting in higher efficiency. </p>

![title](fig2.png)
<p>Fig 2: Red lined circles can be denoted as boundaries of start and end nodes</p>


## Conclusions (Daniel Sedlov 123120712)

Few things in the world of computer science have as ubiquitous a solution as pathfinding does. One look at any common problem such as sorting, encryption, data transfer, or even machine learning will yield dozens if not more of solutions and techniques all suggesting they each be applied at different times in different situations to achieve different results with different compromises. 

While there might be small variations depending on the needs of the user, overall,the core idea is the same. It is fast, efficient, and easy to implement. 

The A* algorithm is a unicorn in that it is quite commonly used as the go-to pathfinding algorithm regardless of the scale, or application. It is widely used in robotics and video games where every millisecond is crucial, and navigational software where finding the most efficient route is the entire function. It is truly a jack of all trades.
